In [295]:
import numpy as np
from tqdm import trange

import os
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import activations

In [1303]:
class Layer():
    def __init__(self, output_shape, input_shape, activation="relu"):
        w_init = tf.keras.initializers.Orthogonal()
        self.weights = tf.Variable(
            initial_value=w_init(shape=(input_shape, output_shape), dtype="float32"),
            trainable=True,
        )
        self.weights = self.weights + np.random.normal(size=(input_shape, output_shape), scale=0.1)

        b_init = tf.keras.initializers.Zeros()
        self.biases = tf.Variable(
            initial_value=b_init(shape=(output_shape,), dtype="float32"), trainable=True
        )

        self.output_shape = output_shape
        self.input_shape = input_shape
        self.size = self.weights.shape
        
        if activation == "relu":
            self.activation = tf.keras.activations.relu
        if activation == "tanh":
            self.activation = tf.keras.activations.tanh
        
    def add_weights(self, weights):
        self.weights = self.weights + weights
        
    def add_biases(self, biases):
        self.biases = self.biases + biases
    
    def extend(self):
        self.weights = np.append(self.weights, np.random.normal(size=(self.input_shape,1)), axis=1)
        self.biases = np.append(self.biases, 0)        
        self.output_shape += 1
        self.size = self.weights.shape
    
    def extend_input(self):
        self.weights = np.append(self.weights, np.random.normal(size=(1,self.output_shape)), axis=0)
        self.input_shape += 1
        self.size = self.weights.shape
        
    def decrease(self):
        self.weights = np.delete(self.weights, -1, axis=0)
        self.biases = np.delete(self.biases, -1)        
        self.output_shape -= 1
        self.size = self.weights.shape
    
    def decrease_input(self):
        self.weights = np.delete(self.weights, -1, axis=1)
        self.input_shape -= 1
        self.size = self.weights.shape
    
    def change_output_size(self, size):
        while self.output_shape != size:
            if self.output_shape > size:
                self.decrease()
            elif self.output_size < size:
                self.extend()
                
    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.weights) + self.biases)
    
    def __repr__(self):
        return "Layer: [shape=" + str(self.weights.shape) + "]"

In [1304]:
class Network():
    def __init__(self, input_shape):
        self.layers = []
        self.input_shape = input_shape
        
    def add_layer(self, size, activation=relu, weights=None, biases=None):
        if len(self.layers) == 0:
            self.layers.append(
                Layer(
                    size,
                    self.input_shape,  
                    #activation=activation, 
                    #weights=weights, 
                    #biases=biases
                )
            )
        else:
            self.layers.append(
                Layer(
                    size, 
                    self.layers[-1].output_shape, 
                    #activation=activation,
                    #weights=weights, 
                    #biases=biases
                )
            )
            
    def predict(self, sample):
        result = sample
        for layer in self.layers:
            result = layer.call(result)
        return np.array(result)
    
    
    def mutate_weights(self, intensity=0.1):
        for layer in self.layers:
            layer.add_weights(np.random.normal(scale=intensity, size=layer.size))
            layer.add_biases(np.random.normal(scale=intensity, size=layer.output_shape))
            
    def mutate_layers(self, propa=0.1):
        for i in range(len(self.layers)-1):
            if np.random.choice([True, False], p=[propa,1-propa]):
                self.layers[i].extend()
                self.layers[i+1].extend_input()
            #elif np.random.choice([True, False], p=[propa,1-propa]):
            #    if self.layers[i].output_shape > 3:
            #        self.layers[i].decrease()
            #        self.layers[i+1].decrease_input()
        
    def mutate_topology(self, propa=0.1, activation="tanh"):
        choice = np.random.choice([0, 1], p=[propa,1-propa])
        # Add a Layer
        if choice == 0:
            if len(self.layers) == 1:
                index = 1
                layer = Layer(input_shape=self.layers[0].output_shape, output_shape=self.layers[0].output_shape, activation=activation)
            else:
                index = np.random.randint(1, len(self.layers))
                layer = Layer(input_shape=self.layers[index-1].output_shape, output_shape=self.layers[index].input_shape, activation=activation)

            self.layers.insert(index, layer)
        # Remove a layer
        #elif choice == 1:
        #    if len(self.layers) > 2:
        #        index = np.random.randint(1, len(self.layers) - 1)
        #        self.layers[index-1].change_output_size(self.layers[index+1].input_size)
        #        del self.layers[index]
            
            
    
    def __repr__(self):
        return "Network: Layers= \n" + str(self.layers)

In [1305]:
net = Network(input_shape=2)
net.add_layer(4)

net.add_layer(7)
net.add_layer(6)
net.add_layer(2)
net.add_layer(3)


#net.mutate_layers(propa=0.1)
net.mutate_topology(propa=1)
net.mutate_layers(propa=1)

print(net)
net.predict(np.ones((8,2)).astype(np.float32))


Network: Layers= 
[Layer: [shape=(2, 5)], Layer: [shape=(5, 8)], Layer: [shape=(8, 8)], Layer: [shape=(8, 7)], Layer: [shape=(7, 3)], Layer: [shape=(3, 3)]]


array([[0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432],
       [0.15864423, 0.        , 0.09909432]], dtype=float32)

In [1306]:
import copy
from keras import backend as K


class NetworkPool():
    def __init__(self, input_shape, output_shape, population_size=100):
        self.networks = []
        for _ in range(population_size):
            
            net = Network(input_shape=input_shape)
            net.add_layer(output_shape, activation="tanh")
            
            self.networks.append(net)            
        #self.networks = np.array(self.networks)
            
    
    def fit(self, data, epochs=100, num_survivors=10, num_children=10, batch_size=32, loss=tf.keras.losses.mean_squared_error):
        X, y = data
        
        batch_start = 0
        
        t = trange(epochs, desc='Loss', leave=True)
        for _ in t:            
            batch_X = np.array(X[batch_start:np.min([batch_start+batch_size, len(X)])]).astype('float32')
            batch_y = np.array(y[batch_start:np.min([batch_start+batch_size, len(y)])]).astype('float32')
            
            batch_start = (batch_start + batch_size) % len(X)
            
            predictions = []
            for network in self.networks:
                predictions.append(network.predict(batch_X))
            
            predictions = np.array(predictions)#.reshape(len(self.networks), -1).astype('float32')
            #print(predictions)
            
            losses = tf.math.reduce_mean(loss(predictions, [batch_y]), axis=-1)

            t.set_description("Population: loss_min={:.4f}, loss_avg={:.4f}".format(np.min(losses), np.mean(losses)))

            idx = np.argsort(losses)

            survivors = [self.networks[x] for x in idx[:num_survivors]] #self.networks[idx[:num_survivors]]
            
            self.best_network = self.networks[idx[0]]
            
            
            # Mutate all surviors and use them as the new networks
            # The best network survives
            self.networks = [self.best_network]
            for survivor1 in survivors:
                #for survivor2 in survivors:
                for _ in range(num_children):
                    #child = survivor1.crossbreed(survivor2, random_mutations=True)
                    child = copy.deepcopy(survivor1)
                    child.mutate_layers()
                    child.mutate_topology()                    
                    child.mutate_weights()
                    self.networks.append(child)
            
            #self.networks = np.array(self.networks)

In [1307]:
X = np.array([[0,0], [0,1], [1,0], [1,1]]).astype(np.float32)
y = np.array([[0.], [1.], [1.], [0.]])

In [1308]:
pool = NetworkPool(input_shape=2, output_shape=1, population_size=200)
pool.fit((X,y), epochs=50)

Population: loss_min=0.0000, loss_avg=0.1067: 100%|██████████| 50/50 [00:03<00:00, 16.66it/s]


In [1309]:
print(pool.best_network.predict(X))
print(pool.best_network)

[[-1.6528554e-04]
 [ 9.9918944e-01]
 [ 9.9695343e-01]
 [-8.8779780e-04]]
Network: Layers= 
[Layer: [shape=(2, 4)], Layer: [shape=(4, 1)]]


In [1310]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

In [1311]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

y_train_onehot = np.zeros((y_train.size, y_train.max()+1))
y_train_onehot[np.arange(y_train.size),y_train] = 1

In [1312]:
print(x_train.shape)
print(y_train_onehot.shape)

(60000, 784)
(60000, 10)


In [1319]:
pool = NetworkPool(input_shape=784,output_shape=10, population_size=100)

In [1320]:
pool.fit((x_train,y_train_onehot), epochs=1000, batch_size=10, num_survivors=4, num_children=4)

Population: loss_min=0.2432, loss_avg=0.4425:  43%|████▎     | 430/1000 [00:13<00:17, 31.95it/s]


KeyboardInterrupt: 

In [1316]:
pool.best_network

Network: Layers= 
[Layer: [shape=(784, 10)], Layer: [shape=(10, 10)], Layer: [shape=(10, 11)], Layer: [shape=(11, 10)], Layer: [shape=(10, 10)], Layer: [shape=(10, 10)]]

In [1317]:
index = [2547,19910,12233,54220]

#print(pool.best_network.predict_all(x_train[index]))
print(np.argmax(pool.best_network.predict_all(x_test), axis=1))
print(y_test)
#print(mae_loss(np.argmax(pool.best_network.predict_all(x_test), axis=1), y_test))

AttributeError: 'Network' object has no attribute 'predict_all'

In [190]:
tf.matmul(np.ones((2,4)), np.ones((4,2)))

<tf.Tensor: shape=(2, 2), dtype=float64, numpy=
array([[4., 4.],
       [4., 4.]])>

In [191]:
np.array(X).shape

(4, 2)

In [611]:
print(tf.keras.losses.mean_squared_error([[1],[23],[3]], [[2],[7],[6]]).numpy())

[  1 256   9]
